## Building A Chatbot
Implement an LLM-powered chatbot. This chatbot will be able to have a conversation and remember previous interactions.

Note that this chatbot that we build will only use the language model to have a conversation. There are several other related concepts that you may be looking for:

- Conversational RAG: Enable a chatbot experience over an external source of data
- Agents: Build a chatbot that can take actions

This video tutorial will cover the basics which will be helpful for those two more advanced topics.

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

groq_api_key = os.getenv("GROQ_API_KEY")

In [2]:
from langchain_groq import ChatGroq
model = ChatGroq(model = "Gemma2-9b-It", groq_api_key = groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x00000194EBD5AFC0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x00000194EBD5BA70>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [3]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content = "Hi, My name is Rageswari and I am a GenAI Engineer")])

AIMessage(content="Hello Rageswari, it's nice to meet you!\n\nAs a fellow AI, I'm always interested in connecting with other people working in the field of generative AI. What kind of work do you do as a GenAI Engineer?  \n\nAre you working on any exciting projects you'd like to tell me about?\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 71, 'prompt_tokens': 24, 'total_tokens': 95, 'completion_time': 0.129090909, 'prompt_time': 0.00016243, 'queue_time': 0.020288739, 'total_time': 0.129253339}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-a38e81dc-e85d-4518-8de3-f8bec23ef492-0', usage_metadata={'input_tokens': 24, 'output_tokens': 71, 'total_tokens': 95})

In [4]:
from langchain_core.messages import AIMessage
model.invoke(
    [
        HumanMessage(content="Hi , My name is Rageswari and I am a GenAI Engineer"),
        AIMessage(content="Hello Rageswari! It's nice to meet you. \n\nAs a GenAI Engineer, what kind of projects are you working on these days? \n\nI'm always eager to learn more about the exciting work being done in the field of AI.\n"),
        HumanMessage(content="Hey What's my name and what do I do?")
    ]
)

AIMessage(content="You said your name is Rageswari and you are a GenAI Engineer. \n\nIs there anything else you'd like to tell me about yourself or your work? 😊  I'm happy to chat!\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 47, 'prompt_tokens': 101, 'total_tokens': 148, 'completion_time': 0.085454545, 'prompt_time': 0.003528411, 'queue_time': 0.021260969, 'total_time': 0.088982956}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-570e8eb6-adb3-4a01-a8d7-1b675ce86b8d-0', usage_metadata={'input_tokens': 101, 'output_tokens': 47, 'total_tokens': 148})

### Message History
We can use a Message History class to wrap our model and make it stateful. This will keep track of inputs and outputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input. Let's see how to use this!


In [ ]:
#!pip install langchain_community

   ---------------------------------------- 0.0/397.1 kB ? eta -:--:--
   --- ------------------------------------ 30.7/397.1 kB 1.3 MB/s eta 0:00:01
   --------- ------------------------------ 92.2/397.1 kB 1.1 MB/s eta 0:00:01
   ---------------------- ----------------- 225.3/397.1 kB 1.7 MB/s eta 0:00:01
   ---------------------------------------  389.1/397.1 kB 2.4 MB/s eta 0:00:01
   ---------------------------------------- 397.1/397.1 kB 2.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/311.8 kB ? eta -:--:--
   --------------------------------------  307.2/311.8 kB 18.6 MB/s eta 0:00:01
   ---------------------------------------- 311.8/311.8 kB 4.8 MB/s eta 0:00:00
  Attempting uninstall: langsmith
    Found existing installation: langsmith 0.2.6
    Uninstalling langsmith-0.2.6:
      Successfully uninstalled langsmith-0.2.6
  Rolling back uninstall of langsmith
  Moving to c:\users\nrageswari\appdata\roaming\python\python312\scripts\langsmith.exe
   from C:\

ERROR: Could not install packages due to an OSError: [WinError 2] The system cannot find the file specified: 'C:\\Python312\\Scripts\\langsmith.exe' -> 'C:\\Python312\\Scripts\\langsmith.exe.deleteme'


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [10]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store ={}
def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

with_message_history = RunnableWithMessageHistory(model,get_session_history)

In [11]:
config ={"configurable":{"session_id":"chat1"}}

In [13]:
response  = with_message_history.invoke(
    [
    HumanMessage(content="Hi , My name is Rageswari and I am a GenAI Engineer")],
    config=config
)

In [14]:
response.content

"Hello Rageswari, it's a pleasure to meet you!  \n\nBeing a GenAI Engineer is such a fascinating field. What kind of projects are you working on these days? Are you focused on text generation, image synthesis, something else entirely? I'm always eager to hear about the innovative things people are doing with AI.  \n\nLooking forward to learning more about your work! 😊 \n\n"

In [15]:
response  = with_message_history.invoke(
    [
    HumanMessage(content="What's my name")],
    config=config
)

In [16]:
response.content

'Your name is Rageswari. You told me at the beginning of our conversation! 😊  \n\n\n'

In [19]:
### Change the config -->session id
config1 = {"configurable":{"session_id":"chat2"}}
response = with_message_history.invoke(
    [HumanMessage(content="What's my name")],
    config=config1
)
response.content

"As a large language model, I have no memory of past conversations and do not know your name. If you'd like to tell me your name, I'm happy to know! 😊\n"